In [ ]:
pip install pandas transformers openpyxl

In [ ]:
import pandas as pd

# Load the uploaded files
sentiment_file_path = '/mnt/data/forex_sentiment_analysisnolabel_modified.xlsx'
aligned_file_path = '/mnt/data/aligned_forex_sentiment_data_corrected.xlsx'

# Reading the data from both files
sentiment_df = pd.read_excel(sentiment_file_path)
aligned_df = pd.read_excel(aligned_file_path)

# Convert the 'Date' in sentiment data to a unified format (YYYY-MM-DD)
sentiment_df['Date'] = pd.to_datetime(sentiment_df['Date'], errors='coerce', utc=True)

# Dropping rows where date could not be parsed
sentiment_df_cleaned = sentiment_df.dropna(subset=['Date'])

# Converting to just the date (YYYY-MM-DD)
sentiment_df_cleaned['Date'] = sentiment_df_cleaned['Date'].dt.date

# Ensure both 'Date' columns are in the same format (datetime64)
aligned_df['Date'] = pd.to_datetime(aligned_df['Date'])
sentiment_df_cleaned['Date'] = pd.to_datetime(sentiment_df_cleaned['Date'])

# Now retry the merge operation
merged_df = pd.merge(aligned_df, sentiment_df_cleaned[['Date', 'articleBody']], on='Date', how='left')

# Handling Case 1 (drop rows where articleBody is NaN)
merged_df_case1 = merged_df.dropna(subset=['articleBody'])

# Clean the 'articleBody' column by converting non-string entries to empty strings
sentiment_df_cleaned['articleBody'] = sentiment_df_cleaned['articleBody'].fillna('').astype(str)

# Handle Case 3 (summarize multiple articles for the same date)
merged_df_case3 = sentiment_df_cleaned.groupby('Date')['articleBody'].apply(lambda x: ' '.join(x)).reset_index()

# Perform the final merge to include the summarized 'articleBody' column into the aligned forex data
merged_df_final = pd.merge(aligned_df, merged_df_case3, on='Date', how='left')

# Fill missing 'articleBody' values by carrying forward the last available news article (Case 2)
# This ensures that rows with missing articles get the last available article.
merged_df_final_filled = merged_df_final.fillna(method='ffill')

# Save the corrected final aligned dataframe to an Excel file
corrected_final_file_path = "/mnt/data/Corrected_Final_Aligned_Forex_Sentiment_Data.xlsx"
merged_df_final_filled.to_excel(corrected_final_file_path, index=False)

# Provide the download link for the user
corrected_final_file_path


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-3-0e02309917e3>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sentiment_df_cleaned['Date'] = sentiment_df_cleaned['Date'].dt.date
<ipython-input-3-0e02309917e3>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentat

Final aligned data with summarization saved.
